In [1]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, classification_report, make_scorer, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV 
import time

train = pd.read_csv("../data/avis/train_noYC_lemma_sent_equil.csv", index_col=0)
y_train = train.etoiles
X_train = train.drop("etoiles", axis = 1)
X_train = X_train[['n_avis','sentiment_norm','longueur_texte']]

test = pd.read_csv('../data/avis/test_noYC_lemma_sent_equil.csv')
y_test = test['etoiles']
X_test = test.drop("etoiles", axis =1)
X_test = X_test[['n_avis','sentiment_norm','longueur_texte']]

X_train.head(2)

,n_avis,sentiment_norm,longueur_texte
5529,1.0,0.727087,95
57109,1.0,-0.989352,73


In [15]:
# Fonctions de vectorisation et de reporting
def model_report(model):
    score = round(model.score(X_test, y_test),2)
    print("train score: ", score)
    y_pred = model.predict(X_test)
    print(pd.crosstab(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    return(score)
def review_vector(df):
    tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # 1000 pour garder l'essentiel, plus?
    vec_text = tfidf_vectorizer.fit_transform(df['text_lemma'])
    print(vec_text[0:5])
    # liste des variables à ajouter
    df = df.drop("text_lemma")
    # Ajouter les variables en format dense, comme le texte vectorisé
    df_tf = hstack([vec_text, csr_matrix(df)])
    from scipy.sparse import csr_matrix, hstack
    return df_tf

# 0 Modelisation de base n_avis, length

# 1. modelisation n avis, sentiment, longeur txt

In [16]:
#X_train, X_test, y_train, y_test = train_test_split(features, target)
model = svm.SVC()
t0 = time()
model.fit(X_train, y_train)
score_sent_svm = model_report(model)

train score:  0.32
col_0      1   2   3   4    5
etoiles                      
1        281  86  78  26   91
2        228  86  82  31  135
3        192  33  76  27  234
4         62   6  65  24  405
5         50   7  54  25  426
              precision    recall  f1-score   support

           1       0.35      0.50      0.41       562
           2       0.39      0.15      0.22       562
           3       0.21      0.14      0.17       562
           4       0.18      0.04      0.07       562
           5       0.33      0.76      0.46       562

    accuracy                           0.32      2810
   macro avg       0.29      0.32      0.26      2810
weighted avg       0.29      0.32      0.26      2810



# 2. Grid SVM n_avis, sentiment, txt length

In [17]:
# definir des hyper paramètres à tester
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']} 

# tester les différents paramètres
grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)
grid.fit(X_train, y_train)
print("best parameters", grid.best_params_)
print("score avec les meilleurs paramètres",round(grid.score(X_test, y_test),2))
score_grid_svm = model_report(grid)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.316 total time=   4.2s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.302 total time=   4.2s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.298 total time=   4.3s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.306 total time=   4.4s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.304 total time=   4.3s
[CV 1/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.364 total time=   4.7s
[CV 2/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.343 total time=   4.9s
[CV 3/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.332 total time=   4.7s
[CV 4/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.358 total time=   4.9s
[CV 5/5] END ......C=0.1, gamma=0.1, kernel=rbf;, score=0.347 total time=   5.0s
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.361 total time=   5.2s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf

# 3. Grid SVM: text TFIDF, n_avis, sent, length

In [23]:
# Vectorization des avis lémmatisés
X_train = review_vector(train)
X_test = review_vector(test)
# definir des hyper paramètres à tester
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']} 

# tester les différents paramètres
grid = GridSearchCV(model, param_grid, refit = True, verbose = 3)
grid.fit(X_train, y_train)
# reporting
score_tfidf_grid_svm = model_report(grid)

# Recap

In [ ]:
scores = [score_base_svm,score_sent_svm, score_grid_svm,score_tfidf_grid_svm]
titles = ["length only svm", "len + sentiment svm","len + sent Grid svm", "len + sent + TFID + grid svm"]
recap = pd.DataFrame({"type de tests": titles,
                      "scores": scores})

recap

NameError: name 'score_tfidf_grid_svm' is not defined

# to try
Renforcer le benchmark
- ~~Tester différents hyper paramètres avec une grid~~

renforcer les features:
- dates sous format jj/mm/yyyy ou autre (mois, jour de la semaine, jour/nuit)?
- ~~nom de l'entreprise dichotomisé?~~
Autre:
- ~~concatener le titre et le texte?~~
- ~~améliorer tfidf en enlevant les stopwords et en lemmatisant~~

Questions :
- Pourquoi est ce que la classification ne s'améliore pas en rajoutant le sentiment, ou les sentiment + tfidf, par rapport à l'entrainement sur la simple longueur du texte?